In [1]:
import keras as K
import os.path

import sys

sys.path.append("../src")
from artstat import util
import numpy as np

Using TensorFlow backend.


In [2]:
import imp
imp.reload(util)

<module 'artstat.util' from '../src/artstat/util.py'>

In [3]:
datadir = "/home/pmilovanov/hg/my/data/gallery-pr/all"

path_train = os.path.join(datadir, "train")
path_test = os.path.join(datadir, "test")

glove = "/home/pmilovanov/data/glove/glove.840B.300d.txt"

In [4]:
words, vocab = util.load_vocab("../vocab.txt", 10000)

In [5]:
emb_matrix = util.load_embeddings(vocab, 300, glove)

In [6]:
print("emb matrix shape", emb_matrix.shape)
print("vocab len", len(vocab))


emb matrix shape (10001, 300)
vocab len 10000


In [56]:
imp.reload(util)

seqlen = 128
stride = 100
X, Y, Xu, Yu = util.load_data_sequences(path_train, vocab, seqlen, stride, numfiles=100)



  0%|          | 64/22860 [00:00<00:35, 639.60it/s]


In [8]:
input_x = K.Input((seqlen,), dtype="int32", name="input_x")
input_xu = K.Input((seqlen,), dtype="float32", name="input_xu")
input_y = K.Input((seqlen,), dtype="int32", name="input_y")
input_yu = K.Input((seqlen,), dtype="float32", name="input_yu")

reshaper_u  = K.layers.Reshape((seqlen,1))
resh_xu = reshaper_u(input_xu)
resh_yu = reshaper_u(input_yu)

dim = emb_matrix.shape[1] + 1

L_emb = K.layers.Embedding(emb_matrix.shape[0], 
                           emb_matrix.shape[1], 
                           input_length=seqlen, 
                           trainable=False,
                           weights=[emb_matrix],
                           name="embedding")


In [9]:
Lemb = L_emb
emb_x = Lemb(input_x)
#emb_y = Lemb(input_y)

In [10]:
Lconcat = K.layers.Concatenate()
concat_x = Lconcat([emb_x, resh_xu])
#concat_y = Lconcat([emb_y, resh_yu])

In [37]:
gru = K.layers.CuDNNGRU(512, return_sequences=True)(concat_x)
gru = K.layers.CuDNNGRU(512, return_sequences=True)(gru)
gru = K.layers.CuDNNGRU(512, return_sequences=True)(gru)
#lstm = K.layers.CuDNNLSTM(128, return_sequences=True)(lstm)
#dense = K.layers.BatchNormalization()(lstm)
#dense = K.layers.Dense(300, activation="relu")(lstm)
#dense = K.layers.BatchNormalization()(dense)
#dense = K.layers.Dense(300, activation="relu")(dense)
#dense = K.layers.BatchNormalization()(dense)
dense = K.layers.Dense(300, activation="relu")(gru)
dense = K.layers.BatchNormalization()(dense)
dense = K.layers.Dense(len(vocab)+2, activation="softmax")(dense)

#dense = K.layers.BatchNormalization()(lstm)
#lstm = K.layers.CuDNNLSTM(256, return_sequences=True)(lstm)
#dense = K.layers.Dense(dim, activation="sigmoid")(dense)
#dense = K.layers.BatchNormalization()(dense)
#dense = K.layers.Dense(dim, activation="sigmoid")(dense)
#dense = K.layers.BatchNormalization()(dense)
#dense = K.layers.Dense(dim, activation="sigmoid")(dense)
#dense = K.layers.BatchNormalization()(dense)
#dense = K.layers.Dense(dim, activation="sigmoid")(dense)



In [38]:
model = K.Model(inputs=[input_x, input_xu],
               outputs=[dense])

In [39]:
Yu.shape

(152720, 128)

In [57]:
batch_size=16

import time

def data_generator(X, Xu, Y, Yu, batch_size=batch_size, vocab_size=10000):
    batches = int(math.floor(X.shape[0]/batch_size))
    while True:
        for i in range(batches):
            t0 = time.time()
            start, end = i*batch_size, (i+1)*batch_size
            bX  = X[start:end]
            bXu = Xu[start:end]
            Yc = K.utils.to_categorical(Y[start:end], num_classes=vocab_size)
            bY = np.concatenate((Yc,
                            np.expand_dims(Yu[start:end], axis=2)), axis=2)
            t1 = time.time() - t0
            #print("time", t1)
            yield [[bX, bXu], [bY]]
        
datagen = data_generator(X, Xu, Y, Yu, vocab_size=len(vocab)+1)

b = next(datagen)

In [58]:
b
zY = np.argmax(b[1][0], axis=2)
#print(b[0][0][0])
#print(zY[0])

print(b[0][0].size, b[0][1].size, b[1][0].size)

2048 2048 20484096


In [61]:
opt = K.optimizers.Adam(lr=0.1)
model.compile(opt, loss='categorical_crossentropy')

In [ ]:
import math

numbatches = int(math.floor(X.shape[0]/batch_size))
model.fit_generator(datagen, steps_per_epoch=numbatches, epochs=5000,
                   use_multiprocessing=False)


Epoch 1/5000
37/37 [==============================] - 8s 228ms/step - loss: 6.6803
Epoch 2/5000
37/37 [==============================] - 8s 228ms/step - loss: 6.5376
Epoch 3/5000
 3/37 [=>............................] - ETA: 7s - loss: 6.5944

In [17]:
predictor = K.Model([input_x, input_xu], [dense])
predictor.compile(opt, loss="categorical_crossentropy")

In [19]:
i = 7
gen = X[i].tolist()
genu = Xu[i].tolist()

tX = np.zeros((1, seqlen), dtype="int32")
tXu = np.zeros((1, seqlen), dtype="float32")
results = []

UNK_IDX = len(words) + 1

iterations = 32
for j in range(iterations):
    tX[0,:] = np.array(gen[-seqlen:], "int32")
    tXu[0,:] = np.array(genu[-seqlen:], "float32")
    
    #print(tX)
    z = predictor.predict([tX, tXu])
    scores=z[0, -1]
    idx = UNK_IDX
    while idx == UNK_IDX:
        idx = np.random.choice(range(len(vocab)+2), p=scores)

    gen.append(idx)
    genu.append(0.0)
    results.append(words[idx])
    sys.stdout.write(results[-1] + " ")
    sys.stdout.flush()
#    print("%s ", results[-1])

IndexError: list index out of range

1.3547268e-16 0.93638355


In [92]:
from sklearn import preprocessing

In [93]:
norm_emb_matrix =  preprocessing.normalize(emb_matrix, axis=1)

In [94]:
zn = preprocessing.normalize(z[0,:], axis=1)


In [95]:
we = zn[-1, :300]

In [96]:
scores = np.matmul(norm_emb_matrix, we)

In [97]:
idx = np.argmax(scores)
print(idx)
word = words[idx]

23


In [98]:
word

'at'

In [84]:
words[2]

'.'